# Calibration using a checkerboard
This is used to recover internal parameters(focal length, optical center, radial distortion) and external parameters (orientation i.e. rotation and translation of the camera wrt a real world system)

In [4]:
import numpy as np
import cv2
import glob
import yaml

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('D:\MS - Phenomics\Datasets\Komatsuna-instance seg-tracking-reconstruction\multi_calibration\*.png')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

h,w = img.shape[:2]

#Camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

#printing the parameters
#print("Camera matrix : \n")
#print(mtx)
#print("dist : \n")
#print(dist)
#print("rvecs : \n")
#print(rvecs)
#print("tvecs :\n")
#print(tvecs)

#writing the parameters to a file
data = {'Camera matrix': np.asarray(mtx).tolist(), 'dist': np.asarray(dist).tolist(), 'rvecs': np.asarray(rvecs).tolist(), 'tvecs': np.asarray(tvecs).tolist()}
with open("calibration.yaml", "w") as f:
    yaml.dump(data, f)